In [ ]:
from datetime import datetime, timedelta
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.offline as pyoff
plt.rcParams['font.family'] = 'Noto Sans Mono CJK TC'
import plotly.graph_objs as go
pyoff.init_notebook_mode()

In [ ]:
tx_data = pd.read_csv("C:\\Users\\GK\\Part4.csv")
list_del = list(tx_data.columns[12:])

for i in list_del:
    del tx_data[i]
del tx_data['客戶自主下單']
del tx_data['機會名稱']
del tx_data['產品代碼']
tx_data.info()

In [ ]:
tx_data['結帳日期'] = pd.to_datetime(tx_data['結帳日期'])
tx_data['結帳日期'].describe()

In [ ]:
tx_data['month']= tx_data['結帳日期'].dt.month 
tx_data['year']= tx_data['結帳日期'].dt.year 
tx_data['yearmonth'] = tx_data['year'].astype(str)+tx_data['month'].astype(str).str.zfill(2)

In [ ]:
tx_data.head(10)

In [ ]:
tx_revenue = tx_data.groupby(['yearmonth'])['總價'].sum().reset_index()
tx_revenue.head()

In [ ]:
plot_data = [
    go.Scatter(
        x=tx_revenue['yearmonth'],
        y=tx_revenue['總價'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Revenue'
    )

In [ ]:
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_revenue['MonthlyGrowth'] = tx_revenue['總價'].pct_change()
tx_revenue.head()

In [ ]:
plot_data = [
    go.Scatter(
#         x=tx_revenue.query("yearmonth < 202009")['yearmonth'],
#         y=tx_revenue.query("yearmonth < 202009")['MonthlyGrowth'],
        x=tx_revenue['yearmonth'],
        y=tx_revenue['MonthlyGrowth'],
    ),
    
    go.Scatter(
#         x=tx_revenue.query("yearmonth < 202009")['yearmonth'],
#         y=tx_revenue.query("yearmonth < 202009")['MonthlyGrowth'],
        x=tx_revenue['yearmonth'],
        y=[0]*tx_revenue.size,
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Growth Rate'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_data.groupby('產品描述')['總價'].sum().sort_values(ascending=False).astype(int)

In [ ]:
tx_ciie = tx_data.query("產品描述=='切削油類'").reset_index(drop=True)
tx_ciie.head()

In [ ]:
tx_monthly_active = tx_ciie.groupby('yearmonth')['帳戶名稱'].nunique().reset_index()
tx_monthly_active.head()

In [ ]:
plot_data = [
    go.Bar(
        x=tx_monthly_active['yearmonth'],
        y=tx_monthly_active['帳戶名稱'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Active Customers'
    )

In [ ]:
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_monthly_active['帳戶名稱'].mean()

In [ ]:
tx_ciie['Quantity'] = tx_ciie['數量']* tx_ciie['規格']

In [ ]:
tx_monthly_sales = tx_ciie.groupby('yearmonth')['Quantity'].sum().reset_index()
tx_monthly_sales.head()

In [ ]:
plot_data = [
    go.Bar(
        x=tx_monthly_sales['yearmonth'],
        y=tx_monthly_sales['Quantity'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Total # of Order'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_monthly_sales['Quantity'].mean()

In [ ]:
tx_monthly_order_avg = tx_ciie.groupby('yearmonth')['總價'].mean().reset_index()
tx_monthly_order_avg.head()

In [ ]:
plot_data = [
    go.Bar(
        x=tx_monthly_order_avg['yearmonth'],
        y=tx_monthly_order_avg['總價'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Order Average'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_monthly_order_avg.總價.mean()

In [ ]:
tx_ciie.info()

# New & Existing Users

In [ ]:
tx_min_purchase = tx_ciie.groupby('帳戶名稱').結帳日期.min().reset_index()

In [ ]:
tx_min_purchase.columns = ['帳戶名稱','MinPurchaseDate']

In [ ]:
tx_min_purchase['month']= tx_min_purchase['MinPurchaseDate'].dt.month 
tx_min_purchase['year']= tx_min_purchase['MinPurchaseDate'].dt.year 
tx_min_purchase['MinPurchaseYearMonth'] = tx_min_purchase['year'].astype(str)+tx_min_purchase['month'].astype(str).str.zfill(2)

In [ ]:
tx_min_purchase

In [ ]:
tx_ciie = pd.merge(tx_ciie, tx_min_purchase, on='帳戶名稱')

In [ ]:
tx_ciie.head()

In [ ]:
tx_ciie['UserType'] = 'New'
tx_ciie.loc[tx_ciie['yearmonth']>tx_ciie['MinPurchaseYearMonth'],'UserType'] = 'Existing'

In [ ]:
tx_ciie.UserType.value_counts()

In [ ]:
tx_user_type_revenue = tx_ciie.groupby(['yearmonth','UserType'])['總價'].sum().reset_index()

In [ ]:
tx_user_type_revenue = tx_user_type_revenue[(tx_user_type_revenue["yearmonth"] != "201701")&(tx_user_type_revenue["yearmonth"] != "202009")]

In [ ]:
# tx_user_type_revenue = tx_user_type_revenue.query("InvoiceYearMonth != 201012 and InvoiceYearMonth != 201112")

In [ ]:
tx_user_type_revenue

In [ ]:
plot_data = [
    go.Scatter(
        x=tx_user_type_revenue.query("UserType == 'Existing'")['yearmonth'],
        y=tx_user_type_revenue.query("UserType == 'Existing'")['總價'],
        name = 'Existing'
    ),
    go.Scatter(
        x=tx_user_type_revenue.query("UserType == 'New'")['yearmonth'],
        y=tx_user_type_revenue.query("UserType == 'New'")['總價'],
        name = 'New'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New vs Existing'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
tx_user_ratio = tx_ciie.query("UserType == 'New'").groupby(['yearmonth'])['帳戶名稱'].nunique()/tx_ciie.query("UserType == 'Existing'").groupby(['yearmonth'])['帳戶名稱'].nunique() 
tx_user_ratio = tx_user_ratio.reset_index()
tx_user_ratio = tx_user_ratio.dropna()

In [ ]:
plot_data = [
    go.Bar(
        x=tx_user_ratio[tx_user_ratio["yearmonth"]!= ( "201702" or "202009")]['yearmonth'],
        y=tx_user_ratio[tx_user_ratio["yearmonth"]!= ( "201702" or "202009")]['帳戶名稱'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New Customer Ratio'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

# Create Signup Data

In [ ]:
tx_min_purchase.head()

In [ ]:
unq_month_year =  tx_min_purchase.MinPurchaseYearMonth.unique()

In [ ]:
unq_month_year

In [ ]:
def generate_signup_date(year_month):
    signup_date = [el for el in unq_month_year if year_month >= el]
    return np.random.choice(signup_date)

In [ ]:
tx_min_purchase['SignupYearMonth'] = tx_min_purchase.apply(lambda row: generate_signup_date(row['MinPurchaseYearMonth']),axis=1)




In [ ]:
tx_min_purchase['InstallYearMonth'] = tx_min_purchase.apply(lambda row: generate_signup_date(row['SignupYearMonth']),axis=1)


In [ ]:
tx_min_purchase.head()

In [ ]:
channels = ['organic','inorganic','referral']

In [ ]:
tx_min_purchase['AcqChannel'] = tx_min_purchase.apply(lambda x: np.random.choice(channels),axis=1)


# Monthly Retention Rate

In [ ]:
tx_ciie.head()

In [ ]:
df_monthly_active = tx_ciie.groupby('yearmonth')['帳戶名稱'].nunique().reset_index()

In [ ]:
tx_user_purchase = tx_ciie.groupby(['帳戶名稱','yearmonth'])['總價'].sum().astype(int).reset_index()

In [ ]:
tx_user_purchase

In [ ]:
tx_user_purchase.總價.sum()

In [ ]:
tx_retention = pd.crosstab(tx_user_purchase['帳戶名稱'], tx_user_purchase['yearmonth']).reset_index()

In [ ]:
tx_retention.head()

In [ ]:
months = tx_retention.columns[2:]

In [ ]:
months

In [ ]:
retention_array = []
for i in range(len(months)-1):
    retention_data = {}
    selected_month = months[i+1]
    prev_month = months[i]
    retention_data['yearmonth'] = int(selected_month)
    retention_data['TotalUserCount'] = tx_retention[selected_month].sum()
    retention_data['RetainedUserCount'] = tx_retention[(tx_retention[selected_month]>0) & (tx_retention[prev_month]>0)][selected_month].sum()
    retention_array.append(retention_data)
    

In [ ]:
tx_retention = pd.DataFrame(retention_array)

In [ ]:
tx_retention.head()

In [ ]:
tx_retention['RetentionRate'] = tx_retention['RetainedUserCount']/tx_retention['TotalUserCount']

In [ ]:
tx_retention

In [ ]:
plot_data = [
    go.Scatter(
        x=tx_retention.query("yearmonth<202009")['yearmonth'],
        y=tx_retention.query("yearmonth<202009")['RetentionRate'],
        name="organic"
    )
    
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Retention Rate'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

# Churn Rate

In [ ]:
tx_retention['ChurnRate'] =  1- tx_retention['RetentionRate']

In [ ]:
plot_data = [
    go.Scatter(
        x=tx_retention.query("yearmonth<202009")['yearmonth'],
        y=tx_retention.query("yearmonth<202009")['ChurnRate'],
        name="organic"
    )
    
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Churn Rate'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

# Cohort Base Retention

In [ ]:
tx_user_purchase.head()

In [ ]:
tx_min_purchase.head()

In [ ]:
tx_retention = pd.crosstab(tx_user_purchase['帳戶名稱'], tx_user_purchase['yearmonth']).reset_index()

In [ ]:
tx_retention = pd.merge(tx_retention,tx_min_purchase[['帳戶名稱','MinPurchaseYearMonth']],on='帳戶名稱')

In [ ]:
tx_retention.head()

In [ ]:
tx_retention.columns

In [ ]:
new_column_names = [ 'm_' + str(column) for column in tx_retention.columns[:-1]]
new_column_names.append('MinPurchaseYearMonth')

In [ ]:
tx_retention.columns = new_column_names

In [ ]:
tx_retention

In [ ]:
months

In [ ]:
retention_array = []
for i in range(len(months)):
    retention_data = {}
    selected_month = months[i]
    prev_months = months[:i]
    next_months = months[i+1:]
    for prev_month in prev_months:
        retention_data[prev_month] = np.nan
        
    total_user_count = tx_retention[tx_retention.MinPurchaseYearMonth ==  selected_month].MinPurchaseYearMonth.count()
    retention_data['TotalUserCount'] = total_user_count
    retention_data[selected_month] = 1 
    
    query = "MinPurchaseYearMonth == {}".format(selected_month)
    

    for next_month in next_months:
        new_query = query + " and {} > 0".format(str('m_' + str(next_month)))
        retention_data[next_month] = np.round(tx_retention.query(new_query)['m_' + str(next_month)].sum()/total_user_count,2)
    retention_array.append(retention_data)
    
    

In [ ]:
tx_retention = pd.DataFrame(retention_array)

In [ ]:
len(months)

In [ ]:
tx_retention.index = months

In [ ]:
tx_retention

In [ ]:
tx_retention.loc["201702",:].sum()